# Fine-tune ruBERT-tiny2

Загружаем библиотеки

In [2]:
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
import nltk
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric, Dataset
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from io import StringIO

Загружаем отобранные статьи: cтатьи из https://ria.ru/export/rss2/archive/index.xml, которые в течение трех суток были изменены и затем проверены дополнительно через Яндекс Speller

In [3]:
with open('datanews.json', encoding="utf-8") as f:
    read_data = f.read()
read_data = read_data.replace('\n][\n', ',\n')
articles = pd.read_json(StringIO(read_data), orient='records')

Формируем предложения из отобранных статей. Второе предложение типа "МОСКВА, 12 янв – РИА Новости" убираем.
В новый DataFrame записываем предложение и признак, что предложения корректные (правильные)

In [4]:
pst = nltk.PunktSentenceTokenizer()
sentences = pd.DataFrame({"target": [], "text": []})
for ind in articles.index:
    sentArticle = pst.tokenize(articles['Article'][ind])
    m = 0
    for s in sentArticle:
        if m != 1:
            sentences.loc[len(sentences.index)] = [1, s]
print("Number of correct sentences: ", len(sentences.index))
sentences.head(3)

Number of correct sentences:  4036


,target,text
0,1,В Госдуме предложили сделать старый Новый год ...
1,1,Председатель союза дачников Подмосковья и депу...
2,1,"""После длинных новогодних праздников людям тяж..."


Добавляем предложения с ошибками. При этом устанавливаем признак, что предложения неправильные

In [5]:
error_list = []
with open('errorsents.txt', encoding="utf-8") as fe:
    error_list = fe.readlines()
for s in error_list:
    sentences.loc[len(sentences.index)] = [0, s]
print("Number of incorrect sentences: ", len(error_list))
sentences.tail(3)

Number of incorrect sentences:  52


,target,text
4085,0,По шести приграничным районам было выпушено не...
4086,0,Веден временный график подачи воды. \n
4087,0,"Уточняется, что целью экипажей был опорный пун..."


Формируем набор для обучения и тестовый набор

In [21]:
train, test = train_test_split(sentences, test_size=0.2, random_state=42)
# train.reset_index()
# test.reset_index()
train_text = train['text'].astype('str')
test_text = test['text'].astype('str')
train_labels = train['target']
test_labels = test['target']


In [7]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [24]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values.tolist(),
    padding = True,
    truncation = True,
    add_special_tokens = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.values.tolist(),
    padding = True,
    truncation = True,
    add_special_tokens = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

In [25]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        # item["labels"] = torch.tensor([self.labels[idx]])
        return item
    
    def __len__(self):
        return len(self.labels)
    
train_dataset = Data(tokens_train, train_labels)
test_dataset = Data(tokens_test, test_labels)

https://www.javatpoint.com/accuracy_score-in-sklearn

In [26]:
def compute_accuracy(Y_true, Y_pred):  
    correctly_predicted = 0  
    # iterating over every label and checking it with the true sample  
    for true_label, predicted in zip(Y_true, Y_pred):  
        if true_label == predicted:  
            correctly_predicted += 1  
    # computing the accuracy score  
    accuracy_score = correctly_predicted / len(Y_true)  
    return accuracy_score  

# Training the model using the Support Vector Classification class of sklearn  
svc = SVC()  
svc.fit(tokens_train['input_ids'], train_labels)  
  
# Computing the accuracy score of the model  
Y_pred = svc.predict(tokens_train['input_ids'])  
score = compute_accuracy(test_labels, Y_pred)  
print(score)  

0.9853300733496333


Функция для расчета метрики. Используется метрику F1, так как классы не сбалансированы:

In [27]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds)
    return {'F1': f1}
# import evaluate
# metric = evaluate.load("accuracy")
# def compute_metrics(pred):
    # logits, labels = pred
    # predictions = np.argmax(logits, axis=-1)
    # return metric.compute(predictions=predictions, references=labels)


Ниже указаны все параметры, которые будут использоваться для обучения:

In [28]:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    # num_train_epochs = 3, #Кол-во эпох для обучения
    num_train_epochs = 16, #Кол-во эпох для обучения
    # per_device_train_batch_size = 8, #Размер пакета для каждого устройства во время обучения
    # per_device_eval_batch_size = 8, #Размер пакета для каждого устройства во время валидации
    # weight_decay =0.01, #Понижение весов
    weight_decay =0.1, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    evaluation_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 1,
    seed=21)

Передача в trainer предообученную модель, tokenizer, данные для обучения, данные для валидации и способ расчета метрики:

In [29]:
trainer = Trainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = test_dataset,
                  compute_metrics = compute_metrics)

Запуск обучения модели:

In [30]:
trainer.train()

  0%|          | 0/6544 [00:00<?, ?it/s]

/tmp/ipykernel_10504/16966540.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ValueError: The model did not return a loss from the inputs, only the following keys: last_hidden_state,pooler_output. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

Сохранение обученной модели:

model_path = "fine-tune-rubert-tiny2"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Написание функции для получения предикта:

In [ ]:
def get_prediction():
    test_pred = trainer.predict(test_dataset)
    labels = np.argmax(test_pred.predictions, axis = -1)
    return labels
pred = get_prediction()

Вывод всей необходимой информации для оценки качества модели:

In [ ]:
print(classification_report(test_labels, pred))
print(f1_score(test_labels, pred))